In [1]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import time
import json
import concurrent.futures 
import re
import random
from lxml import etree
import pymongo
from tqdm import tqdm, trange
import jieba
import importlib
import sys
sys.path.append("CralwerProject") 

In [2]:
import CralwerProject
import CralwerProject.ip_pool as ip_pool
from CralwerProject.parsetools import TextParser, MongoDAO
from CralwerProject.playlistcrawler import PlaylistCrawler
from CralwerProject.songparser import SongParser

## 歌单搜索

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}

In [21]:
valid_ips = ip_pool.get_valid_ips('./data/valid_ips.json')

In [3]:
mongo_config_playlist = {
    'mongo_url': 'mongodb://192.168.206.100:27017/',
    'db_name': 'springTest',
    'collection_name': 'test_play_list'
}

mongo_config_added_playlist = {
    'mongo_url': 'mongodb://192.168.206.100:27017/',
    'db_name': 'springTest',
    'collection_name': 'added_play_list'
}

mongo_config_songdata = {
    'mongo_url': 'mongodb://192.168.206.100:27017/',
    'db_name': 'springTest',
    'collection_name': 'song_data'
}

In [5]:
# 歌单总集合
mongoDAO_playlist = MongoDAO(mongo_config_playlist)
# 新添加的歌单集合
mongoDAO_added_playlist = MongoDAO(mongo_config_added_playlist)
# 歌曲集合
mongoDAO_songdata = MongoDAO(mongo_config_songdata)

In [75]:
playlist_crawler = PlaylistCrawler('./data/valid_ips.json', mongoDAO_added_playlist)
song_parser = SongParser(mongoDAO_songdata)

In [29]:
def get_playlist_from_search(keywords, pages=10):
    playlist_ids = []
    with trange(page) as t:
        for i in range(1, pages+1):
            url = 'https://autumnfish.cn/search?keywords=' + keywords +'&limit=30&type=1000&offset=' + str(i * 30)
             # 随机抽取ip
            ip = random.choice(valid_ips)
            proxies = {}
            if ip.split(":")[0] == 'http':
                proxies = {
                    'http': ip
                }
            else:
                proxies = {
                    'https': ip
                }
            r = rq.get(url=url, headers=headers, proxies=proxies)
            search_dict = json.loads(r.text)
            # 从json中获取歌单id
            for item in search_dict['result']['playlists']:
                playlist_ids.append(item.get('id'))
            t.update(1)
    return playlist_ids

In [27]:
playlist_ids = get_playlist_from_search('恐怖')

In [28]:
len(playlist_ids)

300

In [36]:
type(playlist_ids[0])

int

In [44]:
def process_search_playlist_info(playlist_id_list):
    with trange(len(playlist_id_list)) as t:
        for playlist_id in playlist_id_list:
            # 查询是否已存在该歌单
            result = mongoDAO_playlist.count_documents({'_id':str(playlist_id)})
            if result == 0:
                playlist_crawler.process_playlist_info(str(playlist_id))
            t.update(1)
    # 将新添加的歌单数据同步添加到总集合中 
    added_list = mongoDAO_added_playlist.find()
    for playlist_dict in added_list:
        mongoDAO_playlist.insert_one(playlist_dict)

In [45]:
process_search_playlist_info(playlist_ids)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [02:27<00:00,  2.03it/s]


In [51]:
added_playlist_data = mongoDAO_added_playlist.find()

In [56]:
all_data = song_parser.song_data_concat(added_playlist_data)

  0%|                                                                                          | 0/277 [00:00<?, ?it/s]


In [60]:
all_data[3]

{'_id': 591922,
 'tags': ['轻音乐', '夜晚'],
 'play_lists': '纯音乐-恐怖、诡异、惊悚、压抑 网易云最恐怖的音乐 【日系/恐怖】阴森恐怖却好听的声音 恐怖 恐怖 恐怖惊悚｜孤独的时候听一听 恐怖 恐怖八音盒 蜜汁恐怖音乐 【空灵鬼乐】阴森恐怖 『诡异纯音』恐怖的BGM [阴暗恐怖]鬼畜天线宝宝3同人地图背景音东 恐怖主题音乐 恐怖/悬疑/压抑/紧张（2020.12.9更新） 诡异八音盒，恐怖小说的首选 恐怖故事 心脏病者勿入 恐怖诡异音乐（吓人专用） 恐怖，悬疑，紧张的纯音乐 诡异恐怖音乐 —— 氛围音效 【收集】BGM囤够了我们就做恐怖解谜游戏。 『恐怖纯音乐』『压抑』嘘，你看看你的身后'}

In [61]:
song_parser.process_all_song_data(all_data)

100%|███████████████████████████████████████████████████████████████████████████| 31720/31720 [01:42<00:00, 308.62it/s]


In [76]:
result = mongoDAO_songdata.find({'_id': 591922})

In [78]:
len(result[0]['tags'])

48

In [94]:
result[0]

{'_id': 484058025,
 'tags': ['流行',
  '网络歌曲',
  '综艺',
  '气势',
  '大气',
  '鼓点',
  '看书',
  '学习',
  '游戏',
  '蹦迪',
  'ACG',
  '温柔',
  '电音',
  '动感',
  '经典',
  '散步',
  'DJ',
  '嗨',
  '催泪',
  '治愈',
  '震撼',
  '热血',
  '浪漫',
  '刷题',
  '高能',
  '纯音乐',
  '青春',
  '日系',
  '放松',
  '古风',
  '史诗',
  '温暖',
  '英文',
  '励志',
  '运动',
  '动漫',
  '伤感',
  '战歌',
  '踩点',
  '欧美',
  'BGM',
  '燃',
  '电子',
  '节奏',
  '安静',
  '恋爱',
  '清晨',
  '街舞',
  '舒服',
  '欢快',
  '快乐',
  '优雅',
  '古典',
  '黑暗',
  '钢琴',
  '愉快',
  '伴奏',
  '国风',
  '感动',
  '丧',
  '清新',
  '日本',
  '轻音乐',
  '安眠',
  '恐怖',
  'rap',
  '自习',
  '诡异']}

In [79]:
result = mongoDAO_songdata.find({'tags':{'$in':['恐怖']}})

{ "_id" : 591922, "tags" : [ "浪漫", "影视原声", "ACG", "悲伤", "温柔", "动感", "钢琴", "静心", "节奏", "安静", "学习", "纯音乐", "刷题", "放松", "睡眠", "经典", "史诗", "战歌", "动漫", "BGM", "古典", "蹦迪", "自习", "轻音乐", "治愈", "看书", "伤感", "轻柔", "空灵", "唯美", "英文", "清新", "黑暗", "国风", "日系", "日本", "致郁", "欢快", "安眠", "震撼", "快乐", "感动", "恋爱", "催泪", "吉他", "伴奏", "恐怖", "诡异" ] }


In [6]:
mongoDAO_songdata.count_documents({'tags':{'$all':['恐怖', 'BGM', '惊悚']}})

30198

In [82]:
mongoDAO_songdata.count_documents({'tags':{'$in':['轻松']}})

197838

In [85]:
list_data = mongoDAO_playlist.find()

In [25]:
all_playlist = mongoDAO_playlist.find()

In [13]:
type(all_playlist[1])

dict

In [26]:
filename = 'all_playlist.json'
all_playlist_list = []

In [27]:
for playlist in all_playlist:
    all_playlist_list.append(playlist)

In [31]:
with open(filename, 'w', encoding='utf-8') as fw:       
    json.dump(all_playlist_list,fp=fw, ensure_ascii=False)

In [32]:
with open(filename, 'r', encoding='utf-8') as fr:
    my_playlist = json.load(fr)

In [37]:
my_playlist[6]

{'_id': '5180018018',
 'song_ids': [1491231632,
  1491237295,
  1491231647,
  1491231648,
  1491231659,
  1491231670,
  1491237338,
  1491237340,
  1491231684,
  1491237360,
  1491231712,
  1491237384,
  1491237390,
  1491231764,
  1489266004,
  1489267058,
  1489269048,
  1489269055,
  1489269059,
  1489269061,
  1489267095,
  1489267097,
  1489269066,
  1487528112,
  1487528118,
  1487528121,
  1487528129,
  1487527157,
  1487528134,
  1487528136,
  1487527161,
  1487527164,
  1487528141,
  1485814455,
  1485814456,
  1485815319,
  1485815320,
  1485815324,
  1485815327,
  1485814463,
  1485815332,
  1485814465,
  1485814466,
  1485815349,
  1485815352,
  1485815353,
  1484055473,
  1484054622,
  1484055543,
  1484054677,
  1482148824,
  1482146933,
  1482146943,
  1482148851,
  1481100312,
  1481101180,
  1481100310,
  1481100309,
  1481101179,
  1481100307,
  1481101177,
  1481101175,
  1480361519,
  1480361521,
  1480361525,
  1480360925,
  1480361528,
  1480360927,
  1480361529,


# 歌单用户与歌曲的对应

In [41]:
# 将新添加的歌单数据同步添加到总集合中 
added_list = mongoDAO_added_playlist.find()
for playlist_dict in added_list:
    mongoDAO_playlist.insert_one(playlist_dict)

E11000 duplicate key error collection: springTest.test_play_list index: _id_ dup key: { : "2317794065" }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: springTest.test_play_list index: _id_ dup key: { : "2317794065" }'}
E11000 duplicate key error collection: springTest.test_play_list index: _id_ dup key: { : "818067578" }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: springTest.test_play_list index: _id_ dup key: { : "818067578" }'}
E11000 duplicate key error collection: springTest.test_play_list index: _id_ dup key: { : "3211003706" }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: springTest.test_play_list index: _id_ dup key: { : "3211003706" }'}
E11000 duplicate key error collection: springTest.test_play_list index: _id_ dup key: { : "2112176183" }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: springTest.tes

In [43]:
result = mongoDAO_playlist.find({'_id':'2317794065'})

将歌单作为用户，其订阅数和播放数作为评分  
[歌单id, 歌曲id, 订阅数, 播放数]

In [38]:
# 模拟用户的评分
all_rattings = []

In [39]:
with trange(len(my_playlist)) as t:
    for playlist in my_playlist:
        for song_id in playlist.get('song_ids'):
            playlist_id = playlist.get('_id')
            subscribed_count = playlist.get('subscribedCount')
            play_count = playlist.get('playCount')
            all_rattings.append([playlist_id, song_id, subscribed_count, play_count])
        t.update(1)

100%|██████████████████████████████████████████████████████████████████████████| 64112/64112 [00:13<00:00, 4588.57it/s]


In [40]:
len(all_rattings)

10129381

In [46]:
all_rattings[0]

['4991155974', 28427734, 30512, 4992275]

In [47]:
all_rattings[8]

['4991155974', 28341143, 30512, 4992275]

In [48]:
with open('all_ratting.json', 'w', encoding='utf-8') as fw:       
    json.dump(all_rattings,fp=fw, ensure_ascii=False)

查询具有情感标签的歌曲，该查询的结果集合作为主数据

In [49]:
emotion_tags = ["治愈","安静","放松","震撼","欢快","清新","轻松","浪漫","伤感",
                "温柔","静心","快乐","舒心","悲伤","唯美","感动","催泪","宁静",
                "温暖","丧","欢乐","惊悚","致郁","愉快","诡异","恐怖","黑暗"]

In [53]:
result = mongoDAO_songdata.find({'tags':{'$in':emotion_tags}})

In [55]:
result.collection.find()

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


1158210

In [59]:
result[49]

{'_id': 1416840107,
 'tags': ['综艺',
  '摇滚',
  '华语',
  '快乐',
  '恋爱',
  '青春',
  '浪漫',
  '放松',
  '嗨',
  '节奏',
  '清新',
  '温柔',
  '轻快',
  '漫步',
  'BGM',
  '惬意',
  '运动',
  '经典',
  '温暖',
  '治愈',
  '学习',
  '吉他',
  '爵士',
  '热血',
  '街舞',
  '励志',
  '丧',
  '优雅',
  '伤感',
  '欧美',
  '舒服',
  '轻柔',
  '欢乐',
  '看书',
  '鼓点',
  '古风',
  '游戏',
  '愉快']}

将获取的具有情感标签的歌曲导出成json

In [65]:
filename = 'big_data/all_emotion_songs.json'
all_emotion_songs = []

In [61]:
for song in result:
    all_emotion_songs.append(song)

In [62]:
len(all_emotion_songs)

1158210

In [64]:
with open(filename, 'w', encoding='utf-8') as fw:       
    json.dump(all_emotion_songs,fp=fw, ensure_ascii=False)